# Denison CS181/DA210 Homework 2.g - Step 1

Before you turn this problem in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

In [1]:
import os
import os.path
import pandas as pd

datadir = "publicdata"

---

## Part A: Data normalization

**Q1:** We first consider a dataset based on a tuburculosis dataset from the World Health Organization's ["Global Tuberculosis Report"](https://www.int/tb/country/data/download/en/).

At no point should you rearrange the order of the rows.

To start, read `table6.csv` into a dataframe `df1`.

In [2]:
df1 = pd.read_csv(os.path.join(datadir, "table6.csv"))

# Display the untidy DataFrame
df1

,country,century,yearDigits,rate
0,Afghanistan,19,99,745/19987071
1,Afghanistan,20,0,2666/20595360
2,Brazil,19,99,37737/172006362
3,Brazil,20,0,80488/174504898
4,China,19,99,212258/1272915272
5,China,20,0,213766/1280428583


In [3]:
# Testing cell
assert df1.shape == (6,4)
assert df1.iloc[2,3] == '37737/172006362'

**Q2:** We'll now make this data tidy, using the following steps:

1. Combine `century` and `yearDigits` into one column, `year` (whose values are strings), then drop the two old columns. Use `copy()` to avoid modifying the original data frame. Store the result as `df1a`.
2. Starting from `df1a`, split the column `rate` into two new columns: `cases` (the number before the slash) and `population` (the number after). After you're done, drop `rate`. Store the result as `df1b`.  Again, use `copy()` to avoid modifying either `df1` or `df1a`.

In [4]:
df1a = df1.copy()
df1a["year"] = df1a["century"]*100 + df1a["yearDigits"]
df1a["year"] = df1a["year"].apply(lambda s : str(s))
df1a.drop(["century", "yearDigits"], axis = 1, inplace = True)
df1b = df1a.copy()
df1b["cases"] = df1b["rate"].apply(lambda s : s.strip().split("/")[0])
df1b["population"] = df1b["rate"].apply(lambda s : s.strip().split("/")[1])
df1b.drop("rate", axis = 1, inplace = True)

# Display the tidy DataFrame
df1b

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


In [5]:
# Testing cell
assert df1.shape == (6,4)
assert df1a.shape == (6,3)
assert df1b.shape == (6,4)
assert df1.iloc[2,3] == '37737/172006362'
assert df1a.iloc[3,2] == '2000'
assert df1b.iloc[4,3] == '1272915272'
assert df1b.iloc[0,2] == "745"

---

**Q3:** Read `us_rent_income.csv` into a dataframe (with `GEOID` as the index) named `df_rent_untidy`.

In [6]:
df_rent_untidy = pd.read_csv(os.path.join(datadir, "us_rent_income.csv"))

# Display part of the untidy DataFrame
df_rent_untidy.head()

,GEOID,NAME,variable,estimate,moe
0,1,Alabama,income,24476.0,136.0
1,1,Alabama,rent,747.0,3.0
2,2,Alaska,income,32940.0,508.0
3,2,Alaska,rent,1200.0,13.0
4,4,Arizona,income,27517.0,148.0


In [7]:
# Testing cell
assert df_rent_untidy.shape == (104,5)
assert df_rent_untidy.iloc[0,2] == "income"
assert df_rent_untidy.iloc[1,3] == 747.0

**Q4:** Transform `df_rent_untidy` as needed to make it tidy. Store the result as `df_rent`.

In [12]:
df_rent = df_rent_untidy.pivot_table(index=["GEOID", "NAME"], columns = "variable")
df_rent.reset_index(inplace=True)
df_rent = df_rent.set_index("GEOID")

# Display part of the tidy DataFrame
df_rent.head()

NAME estimate            moe      
variable               income    rent income  rent
GEOID                                             
1            Alabama  24476.0   747.0  136.0   3.0
2             Alaska  32940.0  1200.0  508.0  13.0
4            Arizona  27517.0   972.0  148.0   4.0
5           Arkansas  23789.0   709.0  165.0   5.0
6         California  29454.0  1358.0  109.0   3.0

In [13]:
# Testing cell
assert df_rent.shape == (52,5)
assert df_rent.iloc[0,1] == 24476.0
assert df_rent.iloc[0,2] == 747.0
assert df_rent.iloc[0,3] == 136.0
assert df_rent.iloc[0,4] == 3.0
assert df_rent.iloc[20,1] == 37147.0
assert df_rent.iloc[31,2] == 809.0

---

**Q5:** Consider the data on religions and income, gathered by Pew Research Center and hosted at this link: https://github.com/chendaniely/pandas_for_everyone/blob/master/data/pew.csv.

The data is also available as `"pew.csv"` in the data folder.  In the code cell that follows, read the data into a DataFrame assigned to `df_rel_untidy`.

In [14]:
df_rel_untidy = pd.read_csv(os.path.join(datadir, "pew.csv"))

# Display part of the DataFrame
df_rel_untidy.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116


In [15]:
# Testing cell
assert df_rel_untidy.shape == (18,11)
assert df_rel_untidy.iloc[0,0] == "Agnostic"
assert df_rel_untidy.iloc[0,1] == 27
assert df_rel_untidy.iloc[5,0] == "Evangelical Prot"
assert df_rel_untidy.columns[5] == "$40-50k"
assert df_rel_untidy.iloc[14,8] == 14

**Q6:** This DataFrame is not in tidy form.  Give one of the three tidy-data requirements that it fails to satisfy and explain why.

There are multiple mapping of independent to dependent variables within the same table

**Q7:** Explore the data from the previous question, then **from the data** list the independent variable(s) and the dependent variable(s). Note: this data came from a survey of counting individuals based on their religion and their income category.

independent: religion
dependent: income

**Q8:** Transform as needed to make it tidy.  Store the normalized (tidy) data frame as `df_rel`.

In [17]:
df_rel = df_rel_untidy.melt(id_vars = "religion", var_name = "income", value_name = "value")

# Display part of the tidy DataFrame
df_rel.head()

,religion,income,value
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15


In [18]:
# Testing cell
assert df_rel.shape == (180,3)
assert df_rel.iloc[0,0] == "Agnostic"
assert df_rel.iloc[0,1] == "<$10k"
assert df_rel.iloc[0,2] == 27
assert df_rel.iloc[41,0] == "Evangelical Prot"
assert df_rel.iloc[89,1] == "$40-50k"
assert df_rel.iloc[104,2] == 14

---

---

## Part B

**Q9:** How much time (in minutes/hours) did you spend on this homework assignment?

1 hour

**Q10:** Who was your partner for this assignment?  If you worked alone, say so instead.

I worked alone